# 04. Pinecone Query

Query the index created in notebook 03.

In [22]:
import os
import openai
import pinecone

from tqdm.auto import tqdm
from dotenv import load_dotenv
from IPython.core.display import Markdown

## Configure Environment

In [23]:
load_dotenv()

True

In [24]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_CHAT_MODEL = os.getenv('OPENAI_CHAT_MODEL', 'gpt-3.5-turbo')
OPENAI_EMBED_MODEL = os.getenv('OPENAI_EMBED_MODEL', 'text-embedding-ada-002')

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_INDEX = os.getenv('PINECONE_INDEX', 'openai-dsm100-2022-oct-transcriptions')
PINECONE_ENV = os.getenv('PINECONE_ENV', 'us-east1-gcp')

In [25]:
# configure openai
openai.api_key = OPENAI_API_KEY

# configure pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

## Define Sample Questions

In [26]:
question01 = """
a. AI has a long history with ups and downs, over-optimistic expectations, and the
industrial period. At present, AI is in the era of:

Select ALL statements that apply.

[3 marks]
i. Big data.
ii. Deep learning.
iii. Robotics.
iv. General AI.
"""

In [27]:
question02 = """
f. The more background knowledge an intelligent agent can use – the more intelligent
behaviour it can exhibit. Background knowledge can include:
[3 marks]

Select ALL statements that apply. 

(i). Problem-specific knowledge
(ii). Declarative knowledge
(iii). Procedural knowledge
(iv). A dataset
"""

## Functions

In [28]:
def get_pinecone_index(index_name: str) -> pinecone.index.Index:
    # connect to index
    index = pinecone.Index(index_name)
    
    # view index stats
    print(index.describe_index_stats())  

    return index  

# test the function
index = get_pinecone_index(PINECONE_INDEX)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 100}},
 'total_vector_count': 100}


In [29]:
def get_text_embeddings(text: str) -> list:
    result = openai.Embedding.create(
            input=[text], engine=OPENAI_EMBED_MODEL)

    return result['data'][0]['embedding']

# test the function
#get_text_embeddings(question01)

In [32]:
def query_pinecone_index(index: pinecone.index.Index,
                         query: str,
                         k: int = 5,
                         include_metadata=True) -> list:
    # get embeddings for query
    query_embedding = get_text_embeddings(query)

    # query index
    return index.query(
        vector=query_embedding,
        top_k=k,
        include_metadata=include_metadata
    )

# test the function
#query_pinecone_index(index, question02)

In [76]:
def create_prompt(index: pinecone.index.Index,
                  query: str, 
                  prompt_limit: int = 3750) -> str:
    prompt = ""
    
    # query pinecone for matching transcripts
    pinecone_results = query_pinecone_index(index, query)

    # get relevant contexts
    contexts = [
        x['metadata']['text'] for x in pinecone_results['matches']
    ]

    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n"+
        "Context:\n"
    )
    
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )

    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= prompt_limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )

    return prompt, pinecone_results

# test the function
prompt, context = create_prompt(index, question02)
display(Markdown(prompt))



Answer the question based on the context below.

Context:
[music]In this topic, we discussed how intelligent agents can find solutions and make decisions.  We can see that uninformed or blind search, informed or heuristic search, and we also discuss that domain-specific knowledge can improve decision-making substantially.  The more agent knows and remembers from the past, the more intelligent behavior it can exhibit.  We also talked about meta-learning or meta-reasoning.  When different search strategies are analyzed and compared, in what situation, what strategy would work best.  If you're interested in this topic, please check the recommended literature.  Of course, there are plenty of resources available in our online library and around. 

---

[music]-In this mini lecture, we will talk about knowledge-based agents.  We have already seen in the previous topics that knowledge makes an agent more intelligent, be it domain-specific knowledge, or knowledge of the past experience, what worked well and what did not.  The more an agent knows and remembers, the better decisions it can make.  Agents that are using knowledge for making decisions are called knowledge-based agents.  This component of a knowledge-based agent is called, unsurprisingly, a knowledge base.  Knowledge has to be encoded in a machine processable way, usually, using knowledge representation languages.  What is knowledge?Generally, there is no clear separation in the means between data, knowledge, and information.  It is often the case even in computer science.  It is, well, stating explicitly what we will consider as such in this module.  Data are facts.  Typically, data are encoded as a table.  For example, if a table stores various chemical element sand their freezing and melting points, then an example of a data item or a fact would be, melting point of helium is -272.  2 celsius.  Typically, knowledge is encoded as a set of rules.  For example, if the temperature is below the melting point, then the chemical substance is in a liquid state.  More advanced knowledge can be represented in the form of an executable model.  For example, a metabolic model captures our knowledge about the metabolic pathways, and molecular mechanisms of a particular organism.  Information is a more generic term that combines data and knowledge.  Problems-specific and domain specific knowledge is called background knowledge.  This can be available to a knowledge-based agent from the very beginning

Question: 
f. The more background knowledge an intelligent agent can use – the more intelligent
behaviour it can exhibit. Background knowledge can include:
[3 marks]

Select ALL statements that apply. 

(i). Problem-specific knowledge
(ii). Declarative knowledge
(iii). Procedural knowledge
(iv). A dataset

Answer:

In [57]:
def ask_question(index: pinecone.index.Index,
                  query: str, 
                  prompt_limit: int = 3750):

    # create prompt
    prompt, context = create_prompt(index, query, prompt_limit)

    response = openai.ChatCompletion.create(
        model=OPENAI_CHAT_MODEL,
        messages=[
            { 'role': 'user', 'content': prompt }
        ]
    )

    return response['choices'][0]['message']['content'], context

# test the function
answer, _ = ask_question(index, question02)
display(Markdown('Question:' + question02))
display(Markdown('Answer: ' + answer))

Question:
f. The more background knowledge an intelligent agent can use – the more intelligent
behaviour it can exhibit. Background knowledge can include:
[3 marks]

Select ALL statements that apply. 

(i). Problem-specific knowledge
(ii). Declarative knowledge
(iii). Procedural knowledge
(iv). A dataset


Answer: 

(i). Problem-specific knowledge
(ii). Declarative knowledge
(iii). Procedural knowledge

In [59]:
answer, _ = ask_question(index, question01)
display(Markdown('Question:' + question01))
display(Markdown('Answer: ' + answer))

Question:
a. AI has a long history with ups and downs, over-optimistic expectations, and the
industrial period. At present, AI is in the era of:

Select ALL statements that apply.

[3 marks]
i. Big data.
ii. Deep learning.
iii. Robotics.
iv. General AI.


Answer: i. Big data.
ii. Deep learning.

## Question Definitions

In [93]:
def display_answer(index: pinecone.index.Index,
                   question: str):
   answer, context = ask_question(index, question)

   output = []

   output.append('### Question')
   output.append(question)

   output.append('### Answer')
   output.append(answer)

   output.append('### Context')
   context_list = set(x['metadata']['topic'] + '::' + x['metadata']['title'] for x in context['matches'])
   for item in context_list:
      output.append('- ' + item)

   display(Markdown('\n'.join(output)))

# test the function
display_answer(index, question01)


### Question

a. AI has a long history with ups and downs, over-optimistic expectations, and the
industrial period. At present, AI is in the era of:

Select ALL statements that apply.

[3 marks]
i. Big data.
ii. Deep learning.
iii. Robotics.
iv. General AI.

### Answer
i. Big data.
ii. Deep learning.
### Context
- Module information::Module introduction video
- Topic 1: Introduction::Lecture 2:  Milestones in the history of AI
- Topic 1: Introduction::Lecture 7: State-of-the-art in AI
- Topic 7: Robot Scientist::Lecture 1: Discovery science and the Robot Scientist system

In [50]:
questions = {}

In [94]:
question = """
AI has a long history with ups and downs, over-optimistic expectations, and the
industrial period. At present, AI is in the era of:
[3 marks]

Select ALL statements that apply.
i. Big data.
ii. Deep learning.
iii. Robotics.
iv. General AI.
"""

questions['A_1_a'] = question
display_answer(index, question)

### Question

AI has a long history with ups and downs, over-optimistic expectations, and the
industrial period. At present, AI is in the era of:
[3 marks]

Select ALL statements that apply.
i. Big data.
ii. Deep learning.
iii. Robotics.
iv. General AI.

### Answer
i. Big data.
ii. Deep learning.
### Context
- Module information::Module introduction video
- Topic 1: Introduction::Lecture 2:  Milestones in the history of AI
- Topic 1: Introduction::Lecture 7: State-of-the-art in AI
- Topic 7: Robot Scientist::Lecture 1: Discovery science and the Robot Scientist system

In [95]:
question = """
b. It is important for AI systems to be explainable and interpretable because it:
Select ALL statements that apply.
[3 marks]

i. Gives users confidence in the system.
ii. Is a legal requirement.
iii. Helps in safeguarding against bias.
iv. Enables assessing of system’s robustness and vulnerability. 
"""

questions['A_1_b'] = question
display_answer(index, question)

### Question

b. It is important for AI systems to be explainable and interpretable because it:
Select ALL statements that apply.
[3 marks]

i. Gives users confidence in the system.
ii. Is a legal requirement.
iii. Helps in safeguarding against bias.
iv. Enables assessing of system’s robustness and vulnerability. 

### Answer
i. Gives users confidence in the system.
iii. Helps in safeguarding against bias.
iv. Enables assessing of system’s robustness and vulnerability.
### Context
- Topic 1: Introduction::Lecture 8: Ethical and legal issues
- Topic 1: Introduction::Lecture 10: Explainable AI

In [96]:
question = """
c. Heuristic functions can be generated based on:
Select ALL statements that apply.
[3 marks]

i. Simplified (or relaxed) problems.
ii. Forecasted blind search results.
iii. Previous experience.
iv. Features of the search space.
"""

questions['A_1_c'] = question
display_answer(index, question)

### Question

c. Heuristic functions can be generated based on:
Select ALL statements that apply.
[3 marks]

i. Simplified (or relaxed) problems.
ii. Forecasted blind search results.
iii. Previous experience.
iv. Features of the search space.

### Answer
i. Simplified (or relaxed) problems.
iii. Previous experience.
iv. Features of the search space.
### Context
- Topic 2: Problem solving agents::Lecture 3: Informed search

In [ ]:
question = """
c. Heuristic functions can be generated based on:
Select ALL statements that apply.
[3 marks]

i. Simplified (or relaxed) problems.
ii. Forecasted blind search results.
iii. Previous experience.
iv. Features of the search space.
"""

questions['A_1_c'] = question
display_answer(index, question)

### Question

c. Heuristic functions can be generated based on:
Select ALL statements that apply.
[3 marks]

i. Simplified (or relaxed) problems.
ii. Forecasted blind search results.
iii. Previous experience.
iv. Features of the search space.

### Answer
i. Simplified (or relaxed) problems.
iii. Previous experience.
iv. Features of the search space.
### Context
- Topic 2: Problem solving agents::Lecture 3: Informed search

In [98]:
question = """
e. A genetic algorithm:
Select ALL statements that apply.
[3 marks]

i. Is a global search algorithm
ii. Is a local search algorithm
iii. Is inspired by Charles Darwin’s theory of natural evolution.
iv. Includes three evolutionary processes: selection, gene crossover, and
mutation.
"""

questions['A_1_e'] = question
display_answer(index, question)

### Question

e. A genetic algorithm:
Select ALL statements that apply.
[3 marks]

i. Is a global search algorithm
ii. Is a local search algorithm
iii. Is inspired by Charles Darwin’s theory of natural evolution.
iv. Includes three evolutionary processes: selection, gene crossover, and
mutation.

### Answer
iii. Is inspired by Charles Darwin’s theory of natural evolution.
iv. Includes three evolutionary processes: selection, gene crossover, and mutation.
### Context
- Topic 2: Problem solving agents::Lecture 4: Genetic algorithms

In [99]:
question = """
f. The more background knowledge an intelligent agent can use – the more intelligent
behaviour it can exhibit. Background knowledge can include:
Select ALL statements that apply.
[3 marks]

(i). Problem-specific knowledge
(ii). Declarative knowledge
(iii). Procedural knowledge
(iv). A dataset
"""

questions['A_1_f'] = question
display_answer(index, question)

### Question

f. The more background knowledge an intelligent agent can use – the more intelligent
behaviour it can exhibit. Background knowledge can include:
Select ALL statements that apply.
[3 marks]

(i). Problem-specific knowledge
(ii). Declarative knowledge
(iii). Procedural knowledge
(iv). A dataset

### Answer
(i) Problem-specific knowledge
(ii) Declarative knowledge
(iii) Procedural knowledge
### Context
- Topic 2: Problem solving agents::Lecture: Topic 2 summary
- Topic 3: Knowledge, reasoning and planning::Lecture 1: Knowledge-based agents

In [101]:
question = """
g. An intelligent agent needs to handle uncertainty only when the environment is both:
partially observable and nondeterministic
[3 marks]

Is this statement is True/False?
"""

questions['A_1_g'] = question
display_answer(index, question)

### Question

g. An intelligent agent needs to handle uncertainty only when the environment is both:
partially observable and nondeterministic
[3 marks]

Is this statement is True/False?

### Answer
True
### Context
- Topic 4: Quantifying uncertainty::Lecture 1: Quantifying uncertainty
- Topic 2: Problem solving agents::Lecture 5: Nondeterministic search, partially observable space

In [102]:
question = """
The principle of maximum expected utility (MEU) says that a rational agent should
choose:
Select ONE most appropriate answer.
[3 marks]

(i). The action that maximises the agent’s current utility.
(ii). The action that maximises the agent’s expected utility.
(iii). The utility function that maximises the expected agent’s value.
(iv). The utility function that maximises the current agent’s value.
"""

questions['A_1_h'] = question
display_answer(index, question)

### Question

The principle of maximum expected utility (MEU) says that a rational agent should
choose:
Select ONE most appropriate answer.
[3 marks]

(i). The action that maximises the agent’s current utility.
(ii). The action that maximises the agent’s expected utility.
(iii). The utility function that maximises the expected agent’s value.
(iv). The utility function that maximises the current agent’s value.

### Answer
(ii). The action that maximises the agent’s expected utility.
### Context
- Topic 4: Quantifying uncertainty::Lecture 1: Quantifying uncertainty
- Topic 4: Quantifying uncertainty::Lecture 5: Decision making

In [104]:
question = """
i. The Bayesian approach to the source and status of probability numbers is:
Select ONE most appropriate answer.
[3 marks]

(i). Subjective.
(ii). Objective.
(iii). Frequentist.
(iv). Statistical.
"""

questions['A_1_i'] = question
display_answer(index, question)

### Question

i. The Bayesian approach to the source and status of probability numbers is:
Select ONE most appropriate answer.
[3 marks]

(i). Subjective.
(ii). Objective.
(iii). Frequentist.
(iv). Statistical.

### Answer
(i) Subjective.
### Context
- Topic 4: Quantifying uncertainty::Lecture 1: Quantifying uncertainty
- Topic 4: Quantifying uncertainty::Lecture 3: Bayes rule
- Topic 4: Quantifying uncertainty::Lecture: Introduction to Topic 4

In [105]:
question = """
j. A typical cycle of scientific discovery includes the following steps:
Select ALL statements that apply.
[3 marks]

i. Conducting surveys.
ii. Hypothesis generation.
iii. Deduction – a prediction from the hypothesis about future effects or
experimental results
iv. Hypothesis testing, to verify or disprove predictions.
"""

questions['A_1_j'] = question
display_answer(index, question)

### Question

j. A typical cycle of scientific discovery includes the following steps:
Select ALL statements that apply.
[3 marks]

i. Conducting surveys.
ii. Hypothesis generation.
iii. Deduction – a prediction from the hypothesis about future effects or
experimental results
iv. Hypothesis testing, to verify or disprove predictions.

### Answer
ii. Hypothesis generation.
iv. Hypothesis testing, to verify or disprove predictions.
### Context
- Topic 3: Knowledge, reasoning and planning::Lecture 6: Logical reasoning
- Topic 7: Robot Scientist::Lecture 2: The concept of Robot Scientist
- Topic 8: Robot Scientist part 2::Lecture 1: Hypotheses generation: abduction
- Topic 8: Robot Scientist part 2::Lecture 2: Hypotheses generation: induction

In [106]:
question = """
a) What type of problems are Markov Decision Processes (MDPs) suitable for
modelling?
"""

questions['B_2_a'] = question
display_answer(index, question)

### Question

a) What type of problems are Markov Decision Processes (MDPs) suitable for
modelling?

### Answer
MDPs are suitable for modelling stochastic sequential decision problems, which are problems of making decisions based on a sequence of frames or states, with a probabilistic element and rewards.
### Context
- Topics 9&10: Game Player::Lecture: Formalising the problem

In [107]:
question = """
b) Choose a video game that is significantly different from the Breakout game
seen in the module.
[2 marks]

• Name and briefly describe the game you have chosen.
• What are the control inputs?
• How can you gain points in the game?
• How might an artificial intelligence system perceive the state of the
game? Please do not use the example of screen pixels - think about
the underlying state of the game. 
"""

questions['B_2_b'] = question
display_answer(index, question)

### Question

b) Choose a video game that is significantly different from the Breakout game
seen in the module.
[2 marks]

• Name and briefly describe the game you have chosen.
• What are the control inputs?
• How can you gain points in the game?
• How might an artificial intelligence system perceive the state of the
game? Please do not use the example of screen pixels - think about
the underlying state of the game. 

### Answer


a) This question cannot be answered as there is no information provided to guide the answer.
### Context
- Topics 9&10: Game Player::Lecture: How might we build a game playing AI?
- Topics 9&10: Game Player::Lecture: Game player case study introduction
- Topics 9&10: Game Player::Lecture: Formalising the problem

In [108]:
question = """
c) Name and describe THREE elements of a Markov Decision Process. For
each element, say what it represents from your chosen video game.
[3 marks]
"""

questions['B_2_c'] = question
display_answer(index, question)

### Question

c) Name and describe THREE elements of a Markov Decision Process. For
each element, say what it represents from your chosen video game.
[3 marks]

### Answer
The three elements of a Markov Decision Process are states, actions, and rewards. In a video game, the states could represent different levels or game states. Actions could represent movements or decisions made by the player such as moving left, right or jumping. Rewards represent the points or bonuses earned by the player for completing a specific action or achieving a certain milestone in the game.
### Context
- Topics 9&10: Game Player::Lecture: Formalising the problem

In [114]:
question = """
d) Do you think a Markov Decision Process is a suitable way to model playing
your chosen game? Justify your answer.
[2 marks]
"""

questions['B_2_d'] = question
display_answer(index, question)

### Question

d) Do you think a Markov Decision Process is a suitable way to model playing
your chosen game? Justify your answer.
[2 marks]

### Answer
There is no clear answer to this question as it depends on the specific details of the chosen game and the goals of the modeling. However, based on the information provided in the context, a Markov decision process is a commonly used approach to model stochastic sequential decision problems, and it can be used to build algorithms for decision making. Therefore, it is possible that a Markov decision process could be a suitable way to model playing a game, but more information is needed to make a definitive judgment.
### Context
- Topics 9&10: Game Player::Lecture: Formalising the problem

In [115]:
question = """
e) Reinforcement learning might be described as learning a Q function. If we
are using Q-learning to learn how to play video games, state TWO aspects
that make Q-learning difficult.
[2 marks]
"""

questions['B_2_e'] = question
display_answer(index, question)

### Question

e) Reinforcement learning might be described as learning a Q function. If we
are using Q-learning to learn how to play video games, state TWO aspects
that make Q-learning difficult.
[2 marks]

### Answer


Two aspects that make Q-learning difficult are:
1. The exploration-exploitation trade-off: Q-learning requires exploration of new actions to learn their values, but also needs to exploit learned values to make optimal choices. Finding the right balance between exploration and exploitation can be challenging.
2. The curse of dimensionality: The number of possible states and actions in a game can be extremely large, leading to exponentially growing Q-tables or state-value functions, making it difficult to explicitly compute and learn the optimal values.
### Context
- Topics 9&10: Game Player::Lecture: How might we build a game playing AI?
- Topics 9&10: Game Player::Lecture: Value functions and DQN
- Topics 9&10: Game Player::Lecture: Weekly outro
- Topics 9&10: Game Player::Lecture: The loss function for DQN

In [118]:
question = """
Nowadays AI systems are capable of making autonomous scientific discoveries. In
this module, we considered several AI systems capable of making automated
scientific discoveries, for example, the mobile robotic chemist, the automatic
statistician, the robot scientists Adam and Eve. The research in automated scientific
discovery is flourishing, and the target is to develop systems capable of making
significant scientific discoveries.

a) What are the advantages of automated discovery systems?
[3 marks]
"""

questions['B_3_a'] = question
display_answer(index, question)

### Question

Nowadays AI systems are capable of making autonomous scientific discoveries. In
this module, we considered several AI systems capable of making automated
scientific discoveries, for example, the mobile robotic chemist, the automatic
statistician, the robot scientists Adam and Eve. The research in automated scientific
discovery is flourishing, and the target is to develop systems capable of making
significant scientific discoveries.

a) What are the advantages of automated discovery systems?
[3 marks]

### Answer
The passage does not provide a clear and direct answer to this question, and the information given is focused more on describing examples of automated discovery systems and the progress in this field rather than discussing their advantages. Therefore, it is not possible to provide a proper answer and earn marks for this question.
### Context
- Topic 7: Robot Scientist::Lecture 2: The concept of Robot Scientist
- Topic 7: Robot Scientist::Lecture 3: History of Robot Scientists and future trends

In [121]:
question = """
Nowadays AI systems are capable of making autonomous scientific discoveries. In
this module, we considered several AI systems capable of making automated
scientific discoveries, for example, the mobile robotic chemist, the automatic
statistician, the robot scientists Adam and Eve. The research in automated scientific
discovery is flourishing, and the target is to develop systems capable of making
significant scientific discoveries.

b) What are the current limitations of automated discovery systems? 
[3 marks]
"""

questions['B_3_b'] = question
display_answer(index, question)

### Question

Nowadays AI systems are capable of making autonomous scientific discoveries. In
this module, we considered several AI systems capable of making automated
scientific discoveries, for example, the mobile robotic chemist, the automatic
statistician, the robot scientists Adam and Eve. The research in automated scientific
discovery is flourishing, and the target is to develop systems capable of making
significant scientific discoveries.

b) What are the current limitations of automated discovery systems? 
[3 marks]

### Answer
The passage mentions that while there has been steady progress in the development of automated discovery systems, the discoveries made so far are still on a modest scale. Therefore, one limitation could be the scale and significance of the discoveries made by these systems. Additionally, the target is to develop systems that are capable of significant scientific discoveries, suggesting that the current systems may not be able to achieve this goal yet. Another limitation could be the complexity and range of experiments that a system can perform, as some may be limited to specific domains or experimental setups.
### Context
- Topic 7: Robot Scientist::Lecture 2: The concept of Robot Scientist
- Topic 7: Robot Scientist::Lecture 3: History of Robot Scientists and future trends

In [122]:
question = """
Nowadays AI systems are capable of making autonomous scientific discoveries. In
this module, we considered several AI systems capable of making automated
scientific discoveries, for example, the mobile robotic chemist, the automatic
statistician, the robot scientists Adam and Eve. The research in automated scientific
discovery is flourishing, and the target is to develop systems capable of making
significant scientific discoveries.

c) Describe the functionality of one of the key components of the robot scientist
system, e.g. a hypotheses generator, a reasoning engine, an experiments
planner, or data analysis component. What is required for the functioning of
the selected component? What are its input and output? How does it interact
with other components of the system? Outline the workings of the component.
[4 marks]
"""

questions['B_3_c'] = question
display_answer(index, question)

### Question

Nowadays AI systems are capable of making autonomous scientific discoveries. In
this module, we considered several AI systems capable of making automated
scientific discoveries, for example, the mobile robotic chemist, the automatic
statistician, the robot scientists Adam and Eve. The research in automated scientific
discovery is flourishing, and the target is to develop systems capable of making
significant scientific discoveries.

c) Describe the functionality of one of the key components of the robot scientist
system, e.g. a hypotheses generator, a reasoning engine, an experiments
planner, or data analysis component. What is required for the functioning of
the selected component? What are its input and output? How does it interact
with other components of the system? Outline the workings of the component.
[4 marks]

### Answer
One of the key components of the robot scientist system is the hypotheses generator. Its main function is to infer new hypotheses based on the knowledge and data stored in the system's knowledge base. To function properly, the hypotheses generator requires a well-structured and curated knowledge base that encodes information about the scientific domain it operates in.

The input for the hypotheses generator is the knowledge base, which contains data and information about the scientific domain. It also relies on data generated from previous experiments conducted by the system. The output of the hypotheses generator is a set of inferred hypotheses that are not necessarily true. 

The hypotheses generator interacts with other components of the system, such as the reasoning engine and the experiments planner, to formulate and test the newly generated hypotheses. The reasoning engine evaluates the inferred hypotheses and decides which ones should be tested through experimentation. The experiments planner then designs the necessary experiments and executes them using the system's robotic equipment. 

Overall, the hypotheses generator is a crucial component of the robot scientist system, as it is responsible for generating novel hypotheses that can lead to significant scientific discoveries. It relies on a well-curated knowledge base and works in conjunction with other components of the system to ensure the scientific process is carried out efficiently and effectively.
### Context
- Topic 7: Robot Scientist::Lecture 5: Overview of the toy system: components and their interactions
- Topic 7: Robot Scientist::Lecture 2: The concept of Robot Scientist
- Topic 8: Robot Scientist part 2::Introduction to Topic 8
- Topic 7: Robot Scientist::Introduction to Topic 7

In [123]:
question = """
a) What is a genetic algorithm?
[1 mark]
"""

questions['B_4_a'] = question
display_answer(index, question)

### Question

a) What is a genetic algorithm?
[1 mark]

### Answer
A genetic algorithm is a probabilistic search procedure designed to work on large spaces involving states that can be represented as strings, which reflects the process of natural selection, where the fittest individuals are selected for further reproduction. It includes three evolutionary processes: selection, gene crossover, and mutation, and has five components: initial population, fitness function, selection, crossover, and mutation.
### Context
- Topic 2: Problem solving agents::Lecture 4: Genetic algorithms
- Topics 5&6: Creatures::Lecture: Artificial evolution and genetic algorithms: fitness functions, genetic encoding and manipulation, population models
- Topics 5&6: Creatures::Lecture: Bioinspired computing
- Topics 5&6: Creatures::Lecture: Introduction to Lesson 1

In [124]:
question = """
b) Draw a diagram which illustrates the runtime of a genetic algorithm. Show the
major components of the software and when they are used.
[5 marks]
"""

questions['B_4_b'] = question
display_answer(index, question)

### Question

b) Draw a diagram which illustrates the runtime of a genetic algorithm. Show the
major components of the software and when they are used.
[5 marks]

### Answer


I'm sorry, as an AI language model, I don't have the capability to draw a diagram. However, I can provide you with the major components of a genetic algorithm and when they are used:

1. Initial Population: Starting pool of individuals
2. Fitness Function: Measures the fitness of each individual
3. Selection: Choosing individuals to become parents
4. Crossover: Exchanging genetic information between parents to produce offspring
5. Mutation: Randomly changing genetic information in offspring to introduce diversity
6. Termination: Ending the algorithm when a satisfactory solution is found or the maximum number of iterations is reached  

These components are used throughout the runtime of a genetic algorithm in the following order: 
1. Initial population is created
2. Fitness function is applied to the individuals
3. Selection of parents is performed based on fitness
4. Crossover is applied to produce offspring
5. Mutation is applied to introduce diversity
6. The process repeats until a satisfactory solution is found or the maximum number of iterations is reached.
### Context
- Topic 2: Problem solving agents::Lecture 4: Genetic algorithms
- Topics 5&6: Creatures::Lesson 4 Introduction
- Topics 5&6: Creatures::Lecture: Introduction to Lesson 1

In [126]:
question = """
c) Genetic algorithms employ a design process which is similar to the human
design process. Provide TWO arguments against this statement and TWO
arguments in support of this statement, using details of the design of genetic
algorithms.
[4 marks]
"""

questions['B_4_c'] = question
display_answer(index, question)

### Question

c) Genetic algorithms employ a design process which is similar to the human
design process. Provide TWO arguments against this statement and TWO
arguments in support of this statement, using details of the design of genetic
algorithms.
[4 marks]

### Answer


The context does not provide any information that supports or opposes the statement that genetic algorithms employ a design process similar to the human design process. Therefore, it is not possible to provide two arguments for or against the statement using details of the design of genetic algorithms.
### Context
- Topic 2: Problem solving agents::Lecture 4: Genetic algorithms
- Topics 5&6: Creatures::Lecture: Introduction to Lesson 1
- Topics 5&6: Creatures::Lecture: Lesson 1 summary